# SqueezeNet

SqueezeNet可以说是最早公开的轻量级网络，是在非常著名的论文《SqueezeNet: AlexNet-level accuracy with 50x fewer parametersand<0.5MB model size》[1]中公开的。这个论文的名字也非常简单直接，说明“SqueezeNet”和“AlexNet”具有相同的精确度，但是参数数量却仅为AlexNet的1/50，同时还可以将模型压缩到0.5 MB以下。这可以说是一件非常激动人心的事情了，那么SqueezeNet是怎么做到的？下面让我们逐一解释其中的关键技术。首先是模型压缩。模型压缩的思路非常多，包括奇异值分解、权值剪枝、系数矩阵、霍夫曼编码等各类方法，这些方法都是常用的有损压缩方法，这将会在进阶章节中进行介绍。这些方法的思路就是通过有损的方法，在保持精度的条件下大比例的压缩网络。除了模型压缩，我们还希望能够通过调整网络的内部结构提升网络的准确率。结构调整主要包括宏观结构调整和微观结构调整两种。

## 动机

给定一个参数，少参数的CNN：

1. **越小的模型，训练越快。**更高效的分布式训练：分布式训练的数据并行方法在每个服务器上保留整个模型的副本，处理训练数据集的不同子集。因此，通信开销与模型中的参数数成正比，越小的模型，训练越快。
2. **越小的模型，通信少，频繁更新** Over-the-air update(OTA)，
3. **更容易在嵌入式设备上嵌入**

## Background

![AlexNet](../img/AlexNet.jpg)

AlexNet-level accuracy with 50x fewer parameters and <0.5MB model size[3]



4.8MB->0.47MB
AlexNet-level accuracy with 50x fewer parameters and <0.5MB model size
2300+引用
不同SR 和 不同3*3卷积核数量下


SqueezeNet设计了Fire模块，这个模块包括squeeze和expand操作。squeeze 操作使用一系列1x1卷积，用较小的参数量对特征进行压缩，expand 操作用一系列1x1卷积和3x3卷积进行特征扩容。通过堆叠该模块，网络实现了AlexNet的精度，但参数量是后者的1/510。[3]

首先是模型压缩。模型压缩的思路非常多，包括奇异值分解、权值剪枝、系数矩阵、霍夫曼编码等各类方法，这些方法都是常用的有损压缩方法，这将会在进阶章节中进行介绍。这些方法的思路就是通过有损的方法，在保持精度的条件下大比例的压缩网络。除了模型压缩，我们还希望能够通过调整网络的内部结构提升网络的准确率。结构调整主要包括宏观结构调整和微观结构调整两种。

## 微观结构

所谓微观结构调整一般是指调节网络中的某些组成模块。比如最初的LeNet做手写数字识别使用的是5×5的卷积核，后来的VGGNet使用了3×3的卷积核，甚至在Google的Inception中使用了1×1的卷积核。理论上相同层数的网络，卷积核越小，参数就越少。发展到后来，Google甚至提出了Inception模块，模块中包含了不同维度的滤波器（比如1×1、5×5、1×3、3×1等），然后选用不同的模块组成不同的网络，经过不断的实验，希望通过多个小卷积核替代单一的大卷积核，以带来更好的效果。

## 宏观结构

宏观结构调整其实就是调整网络的层次数和模块的组织方式，这在本章开头已经提到过。关于宏观结构调整有一种与众不同的结构调整方式就是将跨越多个层次的不同层的输出连接到同一个层，比如将来自第6层和第9层的激活值输入到一个新层中，这种思路我们称之为旁路连接，形象一点说这种方法可以发现更多潜在的特征组合形式。

## 核心思路

结合这些设计思路我们来看看SqueezeNet是如何做到模型压缩的。首先SqueezeNet采用了3个主要策略以确保网络中的参数数量更少。

1. 使用1×1的卷积核来替代3×3的卷积核，以减少卷积层的参数数量，这样可以使卷积层参数数量减少9倍。
1. 如果3×3的卷积核无法替代，那么就减少3×3卷积核的通道数量，比如将128×3×3的卷积核减少到32×3×3，那么卷积层参数数量就可以减少4倍。
1. 在卷积层后接入降采样，以得到更大的激活图，也就是说将卷积层的步长设置成大于1，步长越大，卷积运算跳过的数据就越多。在早期进行网络设计时，一般前面的层步长比较大，后面的层步长比较小，导致网络输出的激活图也比较小，影响了分类精度。图6-1是SqueezeNet的核心微观结构FireModule，Fire Module的输入是一组1×1的卷积核，传送到混合了1×1的卷积核和3×3的卷积核的混合扩展层。这里可以调节SqueezeNet的3个超参数，分别是Fire Module的层数、Fire Module中1×1卷积核的数量和3×3卷积核的数量。如图6-1所示


由图6-2我们可以看到除了输出、输出前后接入了卷积层，中间就是指定数量的Fire Module层。除了这些主要方法，SqueezeNet还使用了一些其他小技巧。

1. 由于1×1和3×3的输出激活值的高度和宽度不等，因此输入3×3滤波器的数据会添加上1像素的0边界。
1. 激活层使用ReLU替代Sigmoid。ReLU全称是Rectified Linear Unit，最常见的形式是f(x) =max(0, x)。ReLU训练的收敛速度会比传统的Sigmoid/tanh快很多。
1. 网络中不使用全连接层。全连接层是参数最多的层，需要两层神经元完全连接，如果n个神经元的层和m个神经元的层全连接，将会有m×n个参数。去掉全连接层会减少大量的参数。
1. 训练开始的学习率是0.04，随着训练线性减少学习率。这些小技巧都是经过反复实验与经验的总结，值得我们多多借鉴。




[1]: https://arxiv.org/abs/1602.07360
[2]: https://ai.deepshare.net/detail/v_5ee64a7d02a5f_iff07RH8/3?from=p_5ee641d2e8471_5z8XYfL6&type=6
[3]: https://0809zheng.github.io/2020/06/03/CNN-architecture.htmls
[4]: https://github.com/pytorch/vision/blob/master/torchvision/models/squeezenet.py
[5]: https://github.com/t-vi/AICamera/blob/pytorch_master/Exporting%20Squeezenet%20to%20mobile.ipynbs
[6]: https://weread.qq.com/web/reader/5a5326d0719ecf5f5a52e7ek34132fc02293416a75f431d